In [ ]:

import pandas as pd
import os, glob
import numpy as np
import nibabel as nb
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
from scipy.stats.stats import pearsonr
from scipy.stats import permutation_test
import random

metadata = pd.read_csv(os.path.join(os.getcwd(), '/data/_1.5T_T1-weighted_ICBM_12_11_2022.csv'))

samples = [70,100,120,150]
volume_list_all = sorted(glob.glob(os.path.join(os.getcwd(), '/data/volume', '*.csv')))

segs = ['Left-Thalamus-Proper', 'Left-Caudate', 'Left-Putamen', 'Left-Pallidum',
'Left-Hippocampus', 'Left-Amygdala', 'Left-Accumbens-area',
'Right-Thalamus-Proper', 'Right-Caudate', 'Right-Putamen','Right-Pallidum',
'Right-Hippocampus', 'Right-Amygdala', 'Right-Accumbens-area']

sample = []
stat_neuro = []
stat_local = []


for n in samples:
    volume_list = random.choices(volume_list_all, k=n)



    for seg in segs:
        #Local 1
        age = []
        putamen = []

        for volume in volume_list:
            subject = os.path.split(volume)[1].split('_')[1] + '_' + os.path.split(volume)[1].split('_')[2]

            subj_age = metadata.loc[metadata['Subject'] == subject , 'Age'].values

            age.append(subj_age[0])
            putamen_vol = pd.read_csv(volume, index_col=0).loc[seg, 'neurodesk1']
            putamen.append(putamen_vol)

        data = {'age': age, 'putamen': putamen}
        sys = pd.DataFrame(data)

        #Local 2
        age2 = []
        putamen2 = []

        for volume in volume_list:
            subject = os.path.split(volume)[1].split('_')[1] + '_' + os.path.split(volume)[1].split('_')[2]

            subj_age = metadata.loc[metadata['Subject'] == subject , 'Age'].values

            age2.append(subj_age[0])
            putamen_vol = pd.read_csv(volume, index_col=0).loc[seg, ' neurodesk2']
            putamen2.append(putamen_vol)

        data2 = {'age': age2, 'putamen': putamen2}
        sys2 = pd.DataFrame(data2)


        def statistic(x, y):
            return pearsonr(x, y)[0]
        stat = permutation_test((sys['age'], sys['putamen']), statistic, vectorized=False,
                            permutation_type='pairings',
                            random_state=0)

        stat2 = permutation_test((sys2['age'], sys2['putamen']), statistic, vectorized=False,
                            permutation_type='pairings',
                            random_state=0)        
        stat_neuro.append(stat.statistic - stat2.statistic)     
        sample.append(n)     


    for id, seg in enumerate(segs[:7]):
        #Local 1
        age = []
        putamen = []

        for volume in volume_list:
            subject = os.path.split(volume)[1].split('_')[1] + '_' + os.path.split(volume)[1].split('_')[2]

            subj_age = metadata.loc[metadata['Subject'] == subject , 'Age'].values

            age.append(subj_age[0])
            putamen_vol = pd.read_csv(volume, index_col=0).loc[seg, 'neurodesk1'] + pd.read_csv(volume, index_col=0).loc[segs[id+7], 'neurodesk1']
            putamen.append(putamen_vol)

        data = {'age': age, 'putamen': putamen}
        sys = pd.DataFrame(data)

        #Local 2
        age2 = []
        putamen2 = []

        for volume in volume_list:
            subject = os.path.split(volume)[1].split('_')[1] + '_' + os.path.split(volume)[1].split('_')[2]

            subj_age = metadata.loc[metadata['Subject'] == subject , 'Age'].values

            age2.append(subj_age[0])
            putamen_vol = pd.read_csv(volume, index_col=0).loc[seg, ' neurodesk2']  + pd.read_csv(volume, index_col=0).loc[segs[id+7], ' neurodesk2']
            putamen2.append(putamen_vol)

        data2 = {'age': age2, 'putamen': putamen2}
        sys2 = pd.DataFrame(data2)


        def statistic(x, y):
            return pearsonr(x, y)[0]
        stat = permutation_test((sys['age'], sys['putamen']), statistic, vectorized=False,
                            permutation_type='pairings',
                            random_state=0)

        stat2 = permutation_test((sys2['age'], sys2['putamen']), statistic, vectorized=False,
                            permutation_type='pairings',
                            random_state=0)        
        stat_neuro.append(stat.statistic - stat2.statistic)  
        sample.append(n)           

    for seg in segs:
        #Local 1
        age = []
        putamen = []

        for volume in volume_list:
            subject = os.path.split(volume)[1].split('_')[1] + '_' + os.path.split(volume)[1].split('_')[2]

            subj_age = metadata.loc[metadata['Subject'] == subject , 'Age'].values

            age.append(subj_age[0])
            putamen_vol = pd.read_csv(volume, index_col=0).loc[seg, ' local1']
            putamen.append(putamen_vol)

        data = {'age': age, 'putamen': putamen}
        sys = pd.DataFrame(data)

        #Local 2
        age2 = []
        putamen2 = []

        for volume in volume_list:
            subject = os.path.split(volume)[1].split('_')[1] + '_' + os.path.split(volume)[1].split('_')[2]

            subj_age = metadata.loc[metadata['Subject'] == subject , 'Age'].values

            age2.append(subj_age[0])
            putamen_vol = pd.read_csv(volume, index_col=0).loc[seg, ' local2']
            putamen2.append(putamen_vol)

        data2 = {'age': age2, 'putamen': putamen2}
        sys2 = pd.DataFrame(data2)


        def statistic(x, y):
            return pearsonr(x, y)[0]
        stat = permutation_test((sys['age'], sys['putamen']), statistic, vectorized=False,
                            permutation_type='pairings',
                            alternative='greater',
                            random_state=0)

        stat2 = permutation_test((sys2['age'], sys2['putamen']), statistic, vectorized=False,
                            permutation_type='pairings',
                            alternative='greater', 
                            random_state=0)        
        stat_local.append(stat.statistic - stat2.statistic)  
        sample.append(n)              

    for id, seg in enumerate(segs[:7]):
        #Local 1
        age = []
        putamen = []

        for volume in volume_list:
            subject = os.path.split(volume)[1].split('_')[1] + '_' + os.path.split(volume)[1].split('_')[2]

            subj_age = metadata.loc[metadata['Subject'] == subject , 'Age'].values

            age.append(subj_age[0])
            putamen_vol = pd.read_csv(volume, index_col=0).loc[seg, ' local1'] + pd.read_csv(volume, index_col=0).loc[segs[id+7], ' local1']
            putamen.append(putamen_vol)

        data = {'age': age, 'putamen': putamen}
        sys = pd.DataFrame(data)

        #Local 2
        age2 = []
        putamen2 = []

        for volume in volume_list:
            subject = os.path.split(volume)[1].split('_')[1] + '_' + os.path.split(volume)[1].split('_')[2]

            subj_age = metadata.loc[metadata['Subject'] == subject , 'Age'].values

            age2.append(subj_age[0])
            putamen_vol = pd.read_csv(volume, index_col=0).loc[seg, ' local2']  + pd.read_csv(volume, index_col=0).loc[segs[id+7], ' local2']
            putamen2.append(putamen_vol)

        data2 = {'age': age2, 'putamen': putamen2}
        sys2 = pd.DataFrame(data2)


        def statistic(x, y):
            return pearsonr(x, y)[0]
        stat = permutation_test((sys['age'], sys['putamen']), statistic, vectorized=False,
                            permutation_type='pairings',
                            alternative='greater',
                            random_state=0)

        stat2 = permutation_test((sys2['age'], sys2['putamen']), statistic, vectorized=False,
                            permutation_type='pairings',
                            alternative='greater', 
                            random_state=0)        
        stat_local.append(stat.statistic - stat2.statistic)   
        sample.append(n)    



f_name = 'corr_diff.csv'

f = open(os.path.join(os.getcwd(), f_name), 'w', encoding='utf-8')
f.write('n,neurodesk,local\n')
for n_sample,neuro_diff,local_diff in zip(sample,stat_neuro,stat_local):
    line = []
    line.extend([n_sample,neuro_diff,local_diff])
    line = ",".join([str(x) for x in line])
    f.write(line)
    f.write('\n')

f.close()

In [ ]:
from joypy import joyplot
import matplotlib.pyplot as plt
import pandas as pd
from pandas.api.types import CategoricalDtype

diff = pd.read_csv(os.path.join(os.getcwd(), '/data/corr_diff.csv'), header=0)

sample_dict = {10: 'n=10', 30: 'n=30', 50: 'n=50'}
diff['n'] = diff['n'].map(sample_dict)
n_sample = CategoricalDtype(categories=['n=50','n=30','n=10'], ordered=True)
diff['n'] = diff['n'].astype(n_sample)

fig, ax = joyplot(
    data=diff[[ 'local','neurodesk', 'n']], 
    by='n',
    column=[ 'local','neurodesk'],
    color=['grey', 'orange'],
    fill=True,
    alpha=0.55,
    figsize=(12, 8),
    ylim=(0, 55),
    title='Coefficient difference inter-system by sample size for each subcortical volume - age correlation',
    overlap=0
)

plt.ylabel("Sample size")
plt.xlabel("Coefficient difference")